In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard, LambdaCallback
from keras.applications.resnet50 import ResNet50, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict
from PIL import Image
import skimage
import skimage.io
import jpeg4py as jpeg
from io import BytesIO
import imageio

Using TensorFlow backend.


In [2]:
#parameters
classes = 10
batch_size=100
image_size = 512
train_total = 3500
validation_total = 875

In [3]:
model_resnet = ResNet50(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3))

In [5]:
x = model_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(classes, activation='softmax')(x)

In [10]:
for layer in model_resnet.layers:
    layer.trainable = False

In [12]:
model = Model(inputs=model_resnet.input, outputs=x)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [13]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                    

In [14]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 790600 images belonging to 10 classes.
Found 875 images belonging to 10 classes.


In [1]:
# eight image manipulations(as mentioned in kaggle competition) on training and validation set

In [15]:
def image_aug_valid(image, index):
    Image.LOAD_TRUNCATED_IMAGES = True
    mpimg.LOAD_TRUNCATED_IMAGES = True
    if(index==0):
        res_image = np.uint8(cv2.pow(image/255.,0.8)*255.)
    elif(index==1):
        res_image = np.uint8(cv2.pow(image/255.,1.2)*255.)
    elif(index==2):
        res_image = cv2.resize(image,(0,0),fx=0.5,fy=0.5,interpolation = cv2.INTER_CUBIC)
    elif(index==3):
        res_image = cv2.resize(image,(0,0),fx=0.8,fy=0.8,interpolation = cv2.INTER_CUBIC)    
    elif(index==4):
        res_image = cv2.resize(image,(0,0),fx=1.5,fy=1.5,interpolation = cv2.INTER_CUBIC)
    elif(index==5):
        res_image = cv2.resize(image,(0,0),fx=2.0,fy=2.0,interpolation = cv2.INTER_CUBIC)
    elif(index==6):
        img = Image.fromarray(image)
        out_70_valid = BytesIO()
        img.save(out_70_valid, "JPEG", quality=70)
        res_image = jpeg.JPEG(np.frombuffer(out_70_valid.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_70_valid
    elif(index==7):
        img = Image.fromarray(image)
        out_90_valid = BytesIO()
        img.save(out_90_valid, "JPEG", quality=90)
        res_image = jpeg.JPEG(np.frombuffer(out_90_valid.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_90_valid
    return res_image 

In [16]:
def image_aug(image, index):
    Image.LOAD_TRUNCATED_IMAGES = True
    mpimg.LOAD_TRUNCATED_IMAGES = True
    if(index==0):
        res_image = np.uint8(cv2.pow(image/255.,0.8)*255.)
    elif(index==1):
        res_image = np.uint8(cv2.pow(image/255.,1.2)*255.)
    elif(index==2):
        res_image = cv2.resize(image,(0,0),fx=0.5,fy=0.5,interpolation = cv2.INTER_CUBIC)
    elif(index==3):
        res_image = cv2.resize(image,(0,0),fx=0.8,fy=0.8,interpolation = cv2.INTER_CUBIC)    
    elif(index==4):
        res_image = cv2.resize(image,(0,0),fx=1.5,fy=1.5,interpolation = cv2.INTER_CUBIC)
    elif(index==5):
        res_image = cv2.resize(image,(0,0),fx=2.0,fy=2.0,interpolation = cv2.INTER_CUBIC)
    elif(index==6):
        img = Image.fromarray(image)
        out_70 = BytesIO()
        img.save(out_70, "JPEG", quality=70)
        res_image = jpeg.JPEG(np.frombuffer(out_70.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_70
    elif(index==7):
        img = Image.fromarray(image)
        out_90 = BytesIO()
        img.save(out_90, "JPEG", quality=90)
        res_image = jpeg.JPEG(np.frombuffer(out_90.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_90
    return res_image 

In [17]:
def random_crop(image, crop_size):
    h,w,d = image.shape
    rand_num_h = random.randint(0,h-crop_size)
    rand_num_w = random.randint(0,w-crop_size)
    image_crop = image[rand_num_h:rand_num_h+crop_size,rand_num_w:rand_num_w+crop_size,:]
    return image_crop

In [43]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            
            
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                
                for index,i in enumerate(x_batch):
                    image = jpeg.JPEG('train/' + i).decode()
                    
                    if(len(image.shape)<3):
                        print(_1_)
                        continue
                    rand_num = random.randint(0,7)
                    
                    augmented_image = image_aug(image,rand_num)
                    aug_image_recrop = random_crop(augmented_image,image_size)
                    image_crop_list.append(aug_image_recrop)
                    image_classes_list.append(train_data_dict[j][start+index][1])
                    
                    del image
                    del aug_image_recrop
            
            if(len(image_classes_list)<batch_size-2*min_batch_size):
                break
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]

            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [44]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

In [45]:
def valid_gen(valid_files, valid_classes, batch_size, target_size, imggen):

    valid_data = list(zip(valid_files, valid_classes))
  
    while(True):
        
        for manip_num in range(8):
            for start in range(0, len(valid_data), batch_size):
                image_crop_list = []
                image_classes_list = []
                end = min(start + batch_size, len(valid_data)) 

                x_batch = [valid_data[i][0] for i in range(start,end)]

                for index,i in enumerate(x_batch):
                    image = jpeg.JPEG('validation/' + i).decode() 
                    if(len(image.shape)<3):
                            continue
                    #rand_num = random.randint(0,7)

                    augmented_image = image_aug_valid(image,manip_num)
                    aug_image_recrop = random_crop(augmented_image,image_size)
                    image_crop_list.append(aug_image_recrop)
                    image_classes_list.append(valid_data[start+index][1])

                    del image
                    del aug_image_recrop


                image_crop_list = [x/1. for x in image_crop_list]
                image_crop_list = [preprocess_input(x) for x in image_crop_list]

                x_batch = np.array(image_crop_list, np.float32)
                y_batch = np.array(image_classes_list)         

                yield (x_batch, y_batch)

In [46]:
valid_aug_generator = valid_gen(validation_generator.filenames,to_categorical(validation_generator.classes),batch_size,image_size,validation_datagen) 

In [47]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=1e-5),
             metrics=['accuracy'])

In [50]:
from pushbullet import Pushbullet
pb = Pushbullet('o.KiDDDXPuzV4qKbXh4Lywbgw1tK2oFfq1')

In [51]:
pushbullet_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: pb.push_note("epoch: "+str(epoch),"train_loss: "+str(logs['loss'])+"    val_loss"+str(logs['val_loss'])))

In [52]:
callbacks = [ModelCheckpoint(filepath='resnet_image_aug_full.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
#EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./3-metrics_image_aug_2.csv'),
TensorBoard(log_dir='logs_image_aug', write_graph=True),
            pushbullet_callback]

In [30]:
history = model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = 20*int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=valid_aug_generator,
                    validation_steps= 8*int(np.ceil(validation_total/batch_size)),
                    verbose=1,
                    callbacks=callbacks)

/home/janardhan/anaconda3/envs/dl/lib/python3.5/site-packages/keras/engine/training.py:2033: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/100
699/700 [============================>.] - ETA: 1s - loss: 0.1390 - acc: 0.9602
Epoch 00001: val_loss improved from inf to 0.27694, saving model to resnet_image_aug_full_temp_multiprocess.hdf5
700/700 [==============================] - 824s 1s/step - loss: 0.1390 - acc: 0.9603 - val_loss: 0.2769 - val_acc: 0.9255
Epoch 2/100
 35/700 [>.............................] - ETA: 12:10 - loss: 0.1558 - acc: 0.9500

Process Process-6:
Process Process-3:
Process Process-5:
Process Process-4:
Process Process-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-2:
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/multiprocessing/process.py", line 93, in run
  

KeyboardInterrupt: 

In [48]:
model.load_weights('resnet_image_aug_full.hdf5')

In [49]:
model.evaluate_generator(valid_aug_generator,8*int(np.ceil(validation_total/batch_size)))

[0.19202018937628185, 0.94828572124242783]

In [33]:
# validation predictions for analysis

In [34]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid, axis=1)

In [35]:
validation_generator.class_indices

{'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [36]:
true_positive = np.zeros(10)
false_positive = np.zeros(10)
true_negative = np.zeros(10)
false_negative = np.zeros(10)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [37]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)


true_positive
[ 86.  72.  53.  53.  62.  64.  82.  68.  68.  81.]
false_positive
[ 16.  27.   5.  19.  31.  22.  12.   7.   8.  39.]
false_negative
[ 12.  27.  22.  24.  24.  12.  17.  18.  20.  10.]
Total
[ 98.  99.  75.  77.  86.  76.  99.  86.  88.  91.]


In [3]:
# test set predictions - 512 size

In [22]:
filenames_list = []
images_list = []
for i in os.listdir('test/temp/'):
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))  

In [23]:
images_list = [preprocess_input(x) for x in images_list]

In [24]:
new_array_arr = np.array(images_list)


In [25]:
new_array_arr.shape

(2640, 512, 512, 3)

In [26]:
test_preds = model.predict(new_array_arr)

In [28]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}

In [52]:
predicted_classes = [class_ids[np.argmax(i)] for i in test_preds]

In [58]:
for index,i in enumerate(filenames_list):
    if(i.split('_')[-1]=='manip.tif'):
        predicted_classes[index]=''

In [60]:
submission = pd.DataFrame({'fname':filenames_list,'camera':predicted_classes})
submission.to_csv('submission_resnet_with_aug_trained_fully_512_only_unalt.csv', encoding="utf8", index=False)

In [61]:
from IPython.display import FileLink
FileLink('submission_resnet_with_aug_trained_fully_512_only_unalt.csv')

/home/janardhan/camera_model/submission_resnet_with_aug_trained_fully_512_only_unalt.csv

In [ ]:
# Private LB score - unalt+manip - 95.39